In [24]:
import numpy as np
import pandas as pd
import data_prep 
import TensorRecommend
import os

games_info = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\games_detailed_info2025.csv")
games_info.rename(columns={'id': 'ID'}, inplace=True)
bgg_reviews = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\bgg-26m-reviews.csv")

data_preprocessor = data_prep.DataPreprocessor(games_info, bgg_reviews)
game_info_cut = data_preprocessor.discretize_column(games_info, 'usersrated', n_bins=10, percent=0.05, drop_original=True )
game_info_final = data_preprocessor.discretize_column(game_info_cut, 'yearpublished', n_bins=10, drop_original=True)



In [25]:
dataprep = data_prep.DataPreprocessor(game_info_final, bgg_reviews)
data = dataprep.merge_datasets(featires_to_add = ['yearpublished_bin', 'usersrated_bin', 'minplayers', 'maxplayers'])

In [26]:
data.drop(columns=['Unnamed: 0', 'comment', 'name'], inplace=True)

In [27]:
data.dropna(inplace=True)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17225881 entries, 0 to 17225925
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user               object 
 1   rating             float64
 2   ID                 int64  
 3   yearpublished_bin  int64  
 4   usersrated_bin     int64  
 5   minplayers         int64  
 6   maxplayers         int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 1.0+ GB


# mali model

In [29]:
data_entries, num_users, num_games, num_features, user2idx, game2idx, feature2idx = dataprep.data_to_tuple(data, feature_col=['usersrated_bin', 'minplayers'])

In [30]:
data_train, data_test = dataprep.train_test_split(data_entries, test_size=0.6)

In [ ]:
# sad cemo spremiti data_entries, data_train, data_test, num_users, num_features, user2idx,...
import pickle

to_save = {
    "data": data,
    "num_users": num_users,
    "num_games": num_games,
    "num_features": num_features,
    "user2idx": user2idx,
    "game2idx": game2idx,
    "feature2idx": feature2idx
}

with open("dataset_bundle.pkl", "wb") as f:
    pickle.dump(to_save, f)


MemoryError: 

Idemo na trening!

In [31]:
epochs = 10
k = 5
lambda_ = 0.1
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.1, data_entries=data_train, num_users=num_users, 
                                             num_items=num_games, num_features=num_features)


for epoch in range(epochs):
    tessst.train_epoch(t=epoch+1, initial_lr=0.1, max_norm=5.0)


Epoch Loss: 38852069.3133, NR Loss: 38526562.3691, avg error: 2.3646
Epoch Loss: 36234915.5196, NR Loss: 35909614.3307, avg error: 2.2829
Epoch Loss: 34652240.1555, NR Loss: 34327136.8940, avg error: 2.2320
Epoch Loss: 33485381.1469, NR Loss: 33160470.0121, avg error: 2.1938
Epoch Loss: 32364967.2112, NR Loss: 32040242.8388, avg error: 2.1564
Epoch Loss: 31332224.5086, NR Loss: 31007681.5983, avg error: 2.1214
Epoch Loss: 30483080.2953, NR Loss: 30158714.0523, avg error: 2.0921
Epoch Loss: 29966173.7343, NR Loss: 29641978.8863, avg error: 2.0741
Epoch Loss: 29211778.7406, NR Loss: 28887750.6590, avg error: 2.0476
Epoch Loss: 28062357.2131, NR Loss: 27738492.3288, avg error: 2.0064


In [32]:
tessst.save_model(path = r"C:\Users\dijan\Documents\GitHub\Reccomender-system-for-board-games\model_19_2")

In [41]:
epochs = 5
k = 5
lambda_ = 0.1
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.1, data_entries=data_train, num_users=num_users, 
                                             num_items=num_games, num_features=num_features, loading=True, path=r"C:\Users\dijan\Documents\GitHub\Reccomender-system-for-board-games\model_19_2")
for epoch in range(epochs):
    tessst.train_epoch(t=epoch+1, initial_lr=0.1, max_norm=5.0)


Epoch Loss: 25130048.4500, NR Loss: 24807090.0015, avg error: 1.8974
Epoch Loss: 24637219.3888, NR Loss: 24314401.9093, avg error: 1.8785
Epoch Loss: 24246109.4180, NR Loss: 23923430.2663, avg error: 1.8633
Epoch Loss: 23736513.7526, NR Loss: 23413970.8405, avg error: 1.8434
Epoch Loss: 23400694.8386, NR Loss: 23078285.3174, avg error: 1.8301


In [42]:
tessst.save_model(path = r"C:\Users\dijan\Documents\GitHub\Reccomender-system-for-board-games\model_19_2")

In [43]:
def filter_test_set(train_entries, test_entries):
    """
    Zadrži u test setu samo zapise
    gdje su user i item već u train setu.
    """

    train_users = set()
    train_items = set()

    # izvučemo sve user i item indekse iz traina
    for entry in train_entries:
        u_idx, m_idx, *_ = entry
        train_users.add(u_idx)
        train_items.add(m_idx)

    # filtriramo test
    filtered_test = [
        entry for entry in test_entries
        if entry[0] in train_users and entry[1] in train_items
    ]

    return filtered_test


filtered_test = filter_test_set(data_train, data_test)

In [44]:
epochs = 5
k = 5
lambda_ = 0.1
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.1, data_entries=data_train, num_users=num_users, 
                                             num_items=num_games, num_features=num_features, loading=True, path=r"C:\Users\dijan\Documents\GitHub\Reccomender-system-for-board-games\model_19_2")


In [45]:
import numpy as np

def evaluate_rmse(model, test_entries):
    errors = []

    for entry in test_entries:
        pred = model.predict(entry)
        r = entry[-1]
        errors.append((r - pred) ** 2)

    return np.sqrt(np.mean(errors))

In [46]:
import random

random.seed(42)

n_20 = int(0.2 * len(filtered_test))
subset_20 = random.sample(filtered_test, n_20)

print(evaluate_rmse(tessst, subset_20))

1.8340693983070653


In [47]:
tessst.top_games(tessst.U[0])

[(623, np.float64(10.273031495204277)),
 (879, np.float64(10.113496751900916)),
 (1201, np.float64(9.941953798585118)),
 (516, np.float64(9.936612542977866)),
 (1362, np.float64(9.932361239679302)),
 (1386, np.float64(9.927620394677055)),
 (1040, np.float64(9.885760510278175)),
 (494, np.float64(9.883636024958012)),
 (1091, np.float64(9.870159912489454)),
 (866, np.float64(9.847453616284097))]

In [48]:
def get_game_name(m_idx, idx2game, game_info_df):
    game_id = idx2game[m_idx]
    row = game_info_df.loc[game_info_df["ID"] == game_id]
    if not row.empty:
        return row["name"].values[0]
    return "Unknown"

In [49]:
idx2game = {v: k for k, v in game2idx.items()}

In [50]:
top = tessst.top_games(tessst.U[0])
pretty_results = []

for m_idx, score in top:
    name = get_game_name(m_idx, idx2game, game_info_final)
    pretty_results.append((name, score))

pretty_results

[('Frosthaven', np.float64(10.273031495204277)),
 ('Oathsworn: Into the Deepwood', np.float64(10.113496751900916)),
 ('Ticket to Ride Legacy: Legends of the West', np.float64(9.941953798585118)),
 ('Dune: Imperium – Uprising', np.float64(9.936612542977866)),
 ('Too Many Bones: Undertow', np.float64(9.932361239679302)),
 ('Arkham Horror: The Card Game (Revised Edition)',
  np.float64(9.927620394677055)),
 ('Voidfall', np.float64(9.885760510278175)),
 ('Kingdom Death: Monster', np.float64(9.883636024958012)),
 ('Great Western Trail: New Zealand', np.float64(9.870159912489454)),
 ('Age of Innovation', np.float64(9.847453616284097))]